### Data types for MLLib: Breeze Vectors

In [ ]:
import org.apache.spark.mllib.linalg.{Vector, Vectors}

In [ ]:
val dv: Vector = Vectors.dense(1.0, 0.0, 3.0)

In [ ]:
val nbValues = 3
val indexes = Array(0, 2)
val values = Array(1.0, 3.0)
val sv1: Vector = Vectors.sparse(nbValues, indexes, values)

In [ ]:
sv1.toDense

In [ ]:
val sv2: Vector = Vectors.sparse(3, Seq((0, 1.0), (2, 3.0)))

### LabeledPoints

In [ ]:
import org.apache.spark.mllib.regression.LabeledPoint

In [ ]:
val pos = LabeledPoint(1.0, Vectors.dense(1.0, 0.0, 3.0))

### Create RDD of LabeledPoints from a Dataframe

In [ ]:
:sh wget https://s3-eu-west-1.amazonaws.com/spark-notebook-data/closes.csv -O /tmp/closes.csv

In [ ]:
val c = sparkContext.textFile("/tmp/closes.csv")

In [ ]:
import org.apache.spark.sql.SQLContext
val sqlContext = new SQLContext(sparkContext)
import sqlContext.implicits._ 

In [ ]:
case class Quote(stock:String, date:Long, price:Double) extends java.io.Serializable
object Quote {
  val df = new java.text.SimpleDateFormat("yyyy-MM-dd")
  def parse(d: String): Long = df.parse(d).getTime
}

In [ ]:
val q  = c.map(_.split(",").toList)
           .map{ 
             case List(s, d, p) => Quote(s, Quote.parse(d), p.toDouble)
           }
           .toDF
()

In [ ]:
q.cache()
()

In [ ]:
import org.apache.spark.sql.functions._
val maxDate = q.agg(max("date")).map(_.getAs[Long](0)).first

In [ ]:
val stocks = q.filter($"date" === maxDate && $"price" > 20).select("stock").distinct.sample(false, 0.05, 124L)
              .map(_.getAs[String](0))
              .collect
              .toSet                                                                                                       + "MCHB" // ok ok we're owned! :-D   
val nb = stocks.size
val broadStocks = sparkContext.broadcast(stocks)
()

In [ ]:
val strip = q.filter($"stock" isin (broadStocks.value.toList:_*)).cache()
()

In [ ]:
val dates = strip.groupBy($"date").count.filter($"count" === nb)
()

In [ ]:
val cleanDF = strip.join(dates, dates("date") === q("date")).select($"stock", dates("date"), $"price")
()

In [ ]:
val takeOutMCHB_DF = cleanDF.withColumn("label", when($"stock" startsWith "MCHB", true).otherwise(false)).cache()
()

In [ ]:
LineChart(
  takeOutMCHB_DF.filter($"stock" startsWith "VIDI").orderBy("date").map(r => r.getAs[Long](1) → r.getAs[Double](2)).collect,
  maxPoints=90
)

In [ ]:
val takeOutMCHB = 
  takeOutMCHB_DF.rdd.map { row => 
    (row.getAs[String]("stock"), row.getAs[Long]("date"), row.getAs[Double]("price"), row.getAs[Boolean]("label"))
  }

In [ ]:
val byDateCloses = takeOutMCHB.groupBy( elt => elt._2 )
                                .mapValues{ it => 
                                  val arr = it.toArray
                                  val label = arr.find(_._4).map(_._3).get
                                  val features = arr.filter(!_._4).sortBy(_._1).map(_._3)
                                  label +: features
                                }
                                .values

In [ ]:
val points = byDateCloses.map{ arr => 
                            LabeledPoint(arr(0), Vectors.dense(1d /*add intercept*/ +: arr.drop(1)))
                          }

# Method

Split the data into training and test sets (30% held out for testing)

In [ ]:
import org.apache.spark.mllib.optimization.{LBFGS, LeastSquaresGradient, SquaredL2Updater}
import org.apache.spark.mllib.regression.{LinearRegressionModel}

In [ ]:
val seed = 42L
val Array(trainingData, validationData) = points.randomSplit(Array(0.7, 0.3), seed)

In [ ]:
def createModel(λ:Double = 0.01, trainingData:RDD[LabeledPoint]) = {
  val one = trainingData.first
  val numCorrections = 10
  val convergenceTol = 1e-4
  val maxNumIterations = 100
  val regParam = λ
  val initialWeightsWithIntercept = Vectors.dense(new Array[Double](one.features.size))

  val (weightsWithIntercept, loss) = LBFGS.runLBFGS(
    trainingData.map(lp => (lp.label, lp.features)), // create the RDD[(Double, Vector)]
    new LeastSquaresGradient(),                      // loss function
    new SquaredL2Updater(),
    numCorrections,
    convergenceTol,
    maxNumIterations,
    regParam,
    initialWeightsWithIntercept
  )

  val model = new LinearRegressionModel(weightsWithIntercept, 0.0)
  model
}

In [ ]:
import org.apache.spark.sql.functions._
def predict(model:LinearRegressionModel, validationData:RDD[LabeledPoint]) = {
  
  val labelsAndPredictions = validationData.map { point =>
    val prediction = model.predict(point.features)
    (point.label, prediction)
  }.toDF("point", "prediction")
  
  val withSe = labelsAndPredictions.withColumn("se", pow($"point"-$"prediction", 2))
  
  val testSqrtMSE = scala.math.sqrt(withSe.agg(mean("se")).first.getAs[Double](0))
  
  (model, testSqrtMSE)
}

In [ ]:
def run(λ:Double = 0.01, 
        trainingData:RDD[LabeledPoint], 
        validationData:RDD[LabeledPoint],
        plot:Option[Chart[Seq[(Double, Double)]]]=None) = {
  val (model, r) = predict(createModel(λ, trainingData), validationData)
  plot.foreach(_.addAndApply(Seq((λ, r))))
  (model, r)
}

In [ ]:
sparkContext.setCheckpointDir("/tmp/chckdir")
val tr = trainingData.coalesce(24, true).cache()
tr.checkpoint()
val vd = validationData.coalesce(24, true).cache()
vd.checkpoint()
tr.count
vd.count

Choose arbitrary $\lambda = 0.01$

In [ ]:
val (model, r) = run(0.01, tr, vd)

In [ ]:
val withoutReg = run(0, tr, vd)

Let's try many values of this hyper parameter.

In [ ]:
val plotRuns = LineChart(Seq((0d, withoutReg._2)), maxPoints=1000)

In [ ]:
val testBed = (1d to 20d by 1d) map {i => 10e-8 * math.pow(10, i/2)}
val runs = testBed.map(λ => (λ, run(λ, tr, vd, plot=Some(plotRuns))))

In [ ]:
val bestLambda = runs.minBy(_._2._2)._1
val bestModel = runs.minBy(_._2._2)._2._1
val bestPerf = runs.minBy(_._2._2)._2._2

However, we cannot validate the model against new data. So we only know that this model is best for the given data.

In other words, we have no clue if it over or under fits new data, under the condition that the variability of the universe is not fully captured in the validation test.

> This assumption is viable because the validation test is only (generaly) 30 percent of the known data ; which is itself probably a small fraction of the universe.



Solution: cross-validation!

## Cross Validation

The known data is split in two sets:
* prediction set (`10%`)
* learning set (`90%`)

The learning set is also split in K-Folds.

The validation set will help us to choose the best value for our hyper parameter $\lambda$.

It is crucial to know this hyper parameter as best as possible because it regularizes how the model is built in the family.

In [ ]:
val (predictionSet, learningData) = {
  val Array(predictionSet, learningData) = points.randomSplit(Array(0.1, 0.9), 555L)
  val p = predictionSet.coalesce(24, true).cache()
  p.checkpoint()
  p.count
  val l = learningData.coalesce(24, true).cache()
  l.checkpoint()
  l.count
  (p, l)
}

In [ ]:
val (fold1, fold2, fold3, fold4) = {
  val Array(fold1, fold2, fold3, fold4) = learningData.randomSplit(Array(0.25, 0.25, 0.25, 0.25), 94564L)
  
  val f1 = fold1.coalesce(24, true).cache()
  f1.checkpoint()
  f1.count
  val f2 = fold2.coalesce(24, true).cache()
  f2.checkpoint()
  f2.count
  val f3 = fold3.coalesce(24, true).cache()
  f3.checkpoint()
  f3.count
  val f4 = fold4.coalesce(24, true).cache()
  f4.checkpoint()
  f4.count
  (f1, f2, f3, f4)
}

In [ ]:
val cv1 = (sparkContext.union(fold1, fold2, fold3), fold4)
val cv2 = (sparkContext.union(fold2, fold3, fold4), fold1)
val cv3 = (sparkContext.union(fold3, fold4, fold1), fold2)
val cv4 = (sparkContext.union(fold4, fold1, fold2), fold3)

In [ ]:
val plotCv1 = LineChart(Seq((0d, 0d)), maxPoints=1000, sizes=(300,300))
val plotCv2 = LineChart(Seq((0d, 0d)), maxPoints=1000, sizes=(300,300))
val plotCv3 = LineChart(Seq((0d, 0d)), maxPoints=1000, sizes=(300,300))
val plotCv4 = LineChart(Seq((0d, 0d)), maxPoints=1000, sizes=(300,300))
val allPlot = LineChart(Seq((0d, 0d)), maxPoints=1000, sizes=(600,450))
containerFluid(List(
  List(plotCv1 → 6, plotCv2 → 6), 
  List(plotCv3 → 6, plotCv4 → 6)
)) ++ allPlot

In [ ]:
val cvs = testBed.drop(5).map { λ => 
  val onCv1 = run(λ, cv1._1, cv1._2, plot=Some(plotCv1))._2
  val onCv2 = run(λ, cv2._1, cv2._2, plot=Some(plotCv2))._2
  val onCv3 = run(λ, cv3._1, cv3._2, plot=Some(plotCv3))._2
  val onCv4 = run(λ, cv4._1, cv4._2, plot=Some(plotCv4))._2
  val result = (λ, onCv1+onCv2+onCv3+onCv4)
  allPlot.addAndApply(Seq(result))
  result
}
()

In [ ]:
val bestLambdaCV = cvs.minBy(_._2)._1

In [ ]:
val modelOnInitialTrainingData = createModel(bestLambdaCV, tr)
val (perfOnInitialValidationData, _r) = predict(modelOnInitialTrainingData, vd)
_r

Create model on full learning dataset

In [ ]:
val bestModelCV = createModel(bestLambdaCV, learningData)

Estimate error on test data.

In [ ]:
val cv_r = predict(bestModelCV, predictionSet)._2